# Training covid models
### This notebook is an example usage of how to use the model alongside the covid-data-collector in order to train, evaluate and test the model
#### In this notebook you will find example usages on how to use the core functionalities of the model 

#### Import third party modules, and also the data_collector: covid19_genome and the model module

In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # Uncomment to disable GPU
import glob

from model import Model, DatasetName, load_model, remove_model

__ORIG_WD__ = os.getcwd()

os.chdir(f"{__ORIG_WD__}/../data_collectors/")
from covid19_genome import Covid19Genome

os.chdir(__ORIG_WD__)


2024-01-19 16:49:44.663348: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-19 16:49:44.687758: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-19 16:49:45.108611: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


#### Create a model, or try to load it, if it was already have been created.

In order to use the model, the first thing you have to do is provide it with a dataset (with the help of the data_collector). In the following cell you are provided with an example that create the dataset.

You should note that when you are creating the dataset, you are passing the dataset type. You can obtain the available dataset types in the system by calling the model class function ```get_ds_types()```

In [2]:
model_name = "cov19-1024e"

try:
    print("loading model")
    model = load_model(model_name)
except Exception:
    print("creating model")
    covid19_genome = Covid19Genome()
    lineages = covid19_genome.getLocalLineages(1024)
    lineages.sort()
    dataset = []
    def get_dataset():
        for lineage in lineages:
            dataset.append((lineage, covid19_genome.getLocalAccessionsPath(lineage)))
        return dataset

    portions = {
        DatasetName.trainset.name: 0.8,
        DatasetName.validset.name: 0.1,
        DatasetName.testset.name: 0.1
    }

    dataset = get_dataset()
    model = Model(model_name)
    model.create_datasets(model.get_ds_types()[0], dataset, portions)

loading model


2024-01-19 16:49:45.504387: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-19 16:49:45.519640: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-19 16:49:45.519753: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

After you have created the model, and created its datasets. You can check which neural network structures is available. You can do that by calling the model class function ```get_ml_model_structure()```.

After you see all the ml_model structures available in the system, you can check which hyper parameters are needed to define each and every ml_model structure. This is done by calling the model class function ```get_ml_model_structure_hps()```. The ```get_ml_model_structure_hps()``` will return which hps are required, and what it their type.

In [3]:
print(model.get_ml_model_structures())
print(model.get_ml_model_structure_hps(model.get_ml_model_structures()[-1]))

['VitStructure', 'ConvStructure']
{'convclass': 'required', 'convnet': 'required', 'labels': 'required', 'seq_len': 'required', 'd_model': 'required'}


You can also see which properties help define the current type of dataset by calling to the model class function ```get_ds_props()``` This function could be called only after the dataset have been succesfully created. This function will return the properties of the dataset as well as their values.

In [4]:
print(model.get_ds_props())

{'coverage': 4, 'substitution_rate': 0.005, 'insertion_rate': 0.001, 'deletion_rate': 0.001, 'read_length': 128, 'frag_len': 128, 'num_frags': 256}


A use case of the system with the VitStructure model and the minhash genome datasets (a.k.a. mh_genome_ds).

In the mh_genome_ds the coverage is a dataset property that sets the genome coverage rate.

In the VitStructure, the model_depth is the number of transformer encoders.

In this example use-case these two parameters will help us define a neural network that will be trained on the dataset (with the current coverage rate)

In [5]:
coverage = 4
ml_model_depth = 1
sequencer_instrument = "illumina"
batch_size = 1024
mini_batch_size = 256

In [6]:
sequencer_instrument_to_error_profile_map = {
    "illumina": {
        "substitution_rate": 0.005,
        "insertion_rate": 0.001,
        "deletion_rate": 0.001
    },
    "ont": {
        "substitution_rate": 0.01,
        "insertion_rate": 0.04,
        "deletion_rate": 0.04
    },
    "pacbio": {
        "substitution_rate": 0.005,
        "insertion_rate": 0.025,
        "deletion_rate": 0.025
    },
    "roche": {
        "substitution_rate": 0.002,
        "insertion_rate": 0.01,
        "deletion_rate": 0.01
    }
}

def get_model_name(ml_model_depth, coverage, sequencer_instrument):
    if not sequencer_instrument in sequencer_instrument_to_error_profile_map:
        raise Exception(f"Invalid sequencer instrument: {sequencer_instrument}")
    return f"vit.{ml_model_depth}.{coverage}x.{sequencer_instrument}"

ml_model_name = get_model_name(ml_model_depth, coverage, sequencer_instrument)

#### Adding a new neural network

In this cell we will create an ml_model with the required hps (and also optional) as outputted earlier.

In [7]:
newly_added = True
try:
    model.add_ml_model(ml_model_name, hps={
        "structure": model.get_ml_model_structures()[0],
        "d_model": model.get_ds_props()["frag_len"],
        "seq_len": model.get_ds_props()["num_frags"],
        "d_val": 128,
        "d_key": 128,
        "heads": 8,
        "d_ff": 1024+256,
        "labels":  len(model.get_labels()),
        "activation": "relu",
        "optimizer": {
            "name": "AdamW",
            "params": {
                "learning_rate": 0.001,
            },
        },
        "encoder_repeats": ml_model_depth,
        "regularizer": {
            "name": "l2",
            "params": {
                "l2": 0.0001
            }
        },
        "dropout": 0.2,
    })
except:
    newly_added = False
    print("Model already exists")

Model already exists


In [8]:
# ml_model_class = "efficientnet"
# ml_model_name = "efficient_net_b0"

In [9]:
# model.get_ml_model_structures()

['VitStructure', 'ConvStructure']

In [10]:
# import sys
# try:
#     model.add_ml_model(ml_model_name, hps={
#         "structure": model.get_ml_model_structures()[-1],
#         "convclass": ml_model_class,
#         "convnet": ml_model_name,
#         "seq_len": model.get_ds_props()["num_frags"],
#         "d_model": model.get_ds_props()["frag_len"],
#         "labels":  len(model.get_labels()),
#         "batch_size": batch_size,
#         "mini_batch_size": mini_batch_size,
#     })
# except:
#     # print exception info
#     print(sys.exc_info()[0])
#     print(sys.exc_info()[1])

<class 'Exception'>
ML model efficient_net_b0 already exists.


In [11]:
models = model.list_ml_models()
print(models)

['efficient_net_b0', 'vit.1.4x.illumina']


In [12]:
# if newly_added:
#     assert False, "Please consider doing transfer learning"
# # model.transfer(get_model_name(ml_model_depth, coverage * 2, sequencer_instrument), ml_model_name, False)

In [13]:
# model.change_ml_hps(ml_model_name, {
#     "regularizer": {
#         "name": "l2",
#         "params": {
#             "l2": 0.00005,
#         },
#     },
#     "optimizer": {
#         "name": "AdamW",
#         "params": {
#             "learning_rate": 0.0002,
#         },
#     },
# })

#### Updating the dataset coverage

In [14]:
model.update_ds_props({
    "coverage": coverage,
    } | sequencer_instrument_to_error_profile_map[sequencer_instrument])

#### Setting dataset batch size and training

In [15]:
# mini_batch_size = batch_size
# while(True):
#     try:
#         model.set_ds_batch_size(mini_batch_size)
#         model.train(ml_model_name, epochs=30)
#     except:
#         mini_batch_size = int(mini_batch_size // 2)
#         model.change_ml_hps(ml_model_name, {
#             "batch_size": mini_batch_size,
#         })

In [16]:
model.set_ds_batch_size(mini_batch_size)
model.train(ml_model_name, epochs=30)

/home/zuherj/miniconda3/envs/vital/lib/python3.9/site-packages/keras/src/applications/efficientnet.py:321: UserWarning: This model usually expects 1 or 3 input channels. However, it was passed an input_shape with 4 input channels.
  input_shape = imagenet_utils.obtain_input_shape(


compiled the model with the following metrics: ['categorical_accuracy', 'AUC']


2024-01-19 16:49:49.394413: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-01-19 16:49:49.785955: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Epoch 1/30


2024-01-19 16:49:57.989468: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inefficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


  3/601 [..............................] - ETA: 5:12 - categorical_accuracy: 0.3021 - loss: 2.3557 

2024-01-19 16:50:06.268676: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x9d56f510 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-19 16:50:06.268786: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A5000, Compute Capability 8.6
2024-01-19 16:50:06.280088: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-19 16:50:06.462639: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


601/601 [==============================] - 376s 604ms/step - categorical_accuracy: 0.3731 - loss: 2.1486 - val_categorical_accuracy: 0.3779 - val_loss: 2.1573
Epoch 2/30
601/601 [==============================] - 352s 586ms/step - categorical_accuracy: 0.4482 - loss: 1.8454 - val_categorical_accuracy: 0.4518 - val_loss: 1.8496
Epoch 3/30
601/601 [==============================] - 359s 598ms/step - categorical_accuracy: 0.5057 - loss: 1.6231 - val_categorical_accuracy: 0.5009 - val_loss: 1.6848
Epoch 4/30
 77/601 [==>...........................] - ETA: 4:22 - categorical_accuracy: 0.5321 - loss: 1.5290